In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

### Download de data from the following URLs

(https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet](January-2022)


In [3]:
data = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')


def tweak_data(_df: pd.DataFrame) -> pd.DataFrame:

    _df = _df.assign(
                    **{'duration': (_df.tpep_dropoff_datetime - _df.tpep_pickup_datetime)
                                      .apply(lambda td: td.total_seconds() / 60)}
             )
        
    return _df

df = tweak_data(data)

#df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
#df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
#df = df[(df.duration >= 1) & (df.duration <= 60)]

## Q1 Read the data for January. How many columns are there?
-----
* 16
* 17
* 18
* 19

In [4]:
data.shape

(2463931, 19)

A: 19

## Q2. Computing duration

------------
Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* 41.45
* 46.45
* 51.45
* 56.45

In [5]:
df.duration.std()

46.44530513776499

A: 46.45

## Q3. Dropping outliers
-------------------------
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [4]:
def clean_outliers(_df: pd.DataFrame) -> pd.DataFrame:

    _df = _df.loc[(_df.duration >= 1) & (_df.duration <= 60), :]
        
    return _df

df_cleaned = clean_outliers(df)

In [5]:
df_cleaned.shape[0] / df.shape[0]

0.9827547930522406

A: 98%

Q4. One-hot encoding
----
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [6]:
categorical = ['PULocationID', 'DOLocationID']

df_cleaned = df_cleaned \
                .astype({'PULocationID': 'str', 'DOLocationID': 'str'}) \
                .copy()

train_dicts = df_cleaned[categorical] \
                .to_dict(orient='records') \
                .copy()

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_cleaned[target].values

In [8]:
X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

A: 515

## Q5. Training a model
----
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

* 6.99
* 11.99
* 16.99
* 21.99

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190135964814

A: 6.99

## Q6. Evaluating the model
----
Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

* 7.79
* 12.79
* 17.79
* 22.79

In [10]:
data_val = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

In [11]:
df_val_cleaned = clean_outliers(tweak_data(data_val))

In [12]:
categorical = ['PULocationID', 'DOLocationID']

df_val_cleaned = df_val_cleaned \
                .astype({'PULocationID': 'str', 'DOLocationID': 'str'}) \
                .copy()

val_dicts = df_val_cleaned[categorical] \
                .to_dict(orient='records') \
                .copy()

X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val_cleaned[target].values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786389260883266

A: 7.79